# Multi-task models

## Setting up

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
from pathlib import Path
from typing import Dict, Text
from functools import reduce
import os
import pprint
import tempfile
from typing import Dict, Text
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs
from tensorflow import feature_column
import random
import psutil 
import gc

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Getting data

### Inspecting data

In [4]:
pd.read_csv("../data/hmdata/articles.csv.zip", nrows=5)

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,perceived_colour_value_id,perceived_colour_value_name,perceived_colour_master_id,perceived_colour_master_name,department_no,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,4,Dark,5,Black,1676,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,3,Light,9,White,1676,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,1,Dusty Light,9,White,1676,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,4,Dark,5,Black,1339,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,3,Light,9,White,1339,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [5]:
pd.read_csv("../data/hmdata/customers.csv.zip", nrows=5)

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,NaN,ACTIVE,NONE,54,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


In [6]:
pd.read_csv('../data/hmdata/transactions_train.csv.zip', nrows=5)

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


### Splitting data

In [7]:
def get_train_test_dates(all_data_df, train_days=30, test_days=7):
    
    all_data_df['t_dat'] = pd.to_datetime(all_data_df['t_dat'])
    
    test_end_date = (all_data_df['t_dat'].max() + pd.Timedelta(days=1))
    test_start_date = test_end_date - pd.Timedelta(days=test_days)
    train_start_date = test_start_date - pd.Timedelta(days=train_days)
    
    print('train start date:', train_start_date)
    print('test start date:', test_start_date)
    print('test end date:', test_end_date)
    
    return train_start_date, test_start_date, test_end_date

In [8]:
def get_df(
    start_date,
    end_date
):
    df = all_data_df.loc[
        (all_data_df['t_dat'] >= start_date)
        & (all_data_df['t_dat'] < end_date),
        :
    ]
    
    return df

def get_train_df():
    train_start_date, test_start_date, test_end_date = get_train_test_dates(all_data_df)
    train_df = get_df(train_start_date, test_start_date)
    
    return train_df

def get_test_df():
    train_start_date, test_start_date, test_end_date = get_train_test_dates(all_data_df)
    test_df = get_df(test_start_date, test_end_date)
    
    return test_df

### Cleaning data

In [9]:
def clean_article_id(
    df
):
    df = df.copy()
    df['article_id'] = df['article_id'].astype(str)
    # zfill adds zeros (0) at the beginning of the string, until it reaches the specified length
    df['article_id'] = df['article_id'].apply(lambda x: x.zfill(10))
    
    return df

### Munging data

In [10]:
def add_article_features(
    df
):
    df = df.join(
        article_df.set_index('article_id'),
        on='article_id'
    )
    
    return df

In [11]:
def add_aggregated_customer_features(
    df,
    use_train_df
):
    if use_train_df:
        feature_df = train_df.copy()
    else:
        feature_df = df.copy()
    
    dataframes = [
        feature_df.groupby("customer_id", as_index=False).article_id.nunique(), #unique articles
        feature_df.groupby("customer_id", as_index=False).t_dat.nunique(), #days active
        feature_df.groupby("customer_id", as_index=False).size(), #total purchases
        feature_df.groupby("customer_id", as_index=False).price.min(), #min price
        feature_df.groupby("customer_id", as_index=False).price.max(), #max price
        feature_df.groupby("customer_id", as_index=False).price.mean(), #mean price
        feature_df.groupby("customer_id", as_index=False).product_group_name.nunique(), #num unique product groups
        feature_df.groupby("customer_id", as_index=False).department_name.nunique(), #num unique department_name
        feature_df.groupby("customer_id", as_index=False).section_name.nunique(), #num unique section_name
    ]
    
    agg_df = reduce(lambda left,right: pd.merge(left, right, on=['customer_id'], how='outer'), dataframes)
    
    agg_df.columns = [
        'customer_id',
        'nArticles',
        'nDays',
        'nPurchases',
        'minPrice',
        'maxPrice',
        'meanPrice',
        'nPGroups',
        'nDept',
        'nSect'
    ]
    
    df = df.merge(agg_df, on='customer_id', how='outer')
    
    return df

In [12]:
def add_features(
    df,
    use_train_df
):
    df = add_article_features(df)
    df = add_aggregated_customer_features(df, use_train_df)
    
    return df

### Preparing data

In [13]:
article_df = pd.read_csv("../data/hmdata/articles.csv.zip")
article_df = clean_article_id(article_df)

In [14]:
customer_df = pd.read_csv("../data/hmdata/customers.csv.zip")

In [15]:
all_data_df = pd.read_csv('../data/hmdata/transactions_train.csv.zip')

In [16]:
train_df = get_train_df()
train_df = clean_article_id(train_df)
train_df = add_features(train_df, use_train_df=False)

train_df.head()

train start date: 2020-08-17 00:00:00
test start date: 2020-09-16 00:00:00
test end date: 2020-09-23 00:00:00


,t_dat,customer_id,article_id,price,sales_channel_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,perceived_colour_value_id,perceived_colour_value_name,perceived_colour_master_id,perceived_colour_master_name,department_no,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc,nArticles,nDays,nPurchases,minPrice,maxPrice,meanPrice,nPGroups,nDept,nSect
0,2020-08-17,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,0903326005,0.025407,2,903326,Bert Relaxed Printed Tee,255,T-shirt,Garment Upper body,1010014,Placement print,12,Light Beige,1,Dusty Light,11,Beige,5882,Jersey Fancy,F,Menswear,3,Menswear,55,Contemporary Street,1005,Jersey Fancy,Round-necked T-shirt in heavy cotton jersey wi...,14,2,22,0.013542,0.050831,0.032341,3,8,9
1,2020-08-17,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,0903326005,0.025407,2,903326,Bert Relaxed Printed Tee,255,T-shirt,Garment Upper body,1010014,Placement print,12,Light Beige,1,Dusty Light,11,Beige,5882,Jersey Fancy,F,Menswear,3,Menswear,55,Contemporary Street,1005,Jersey Fancy,Round-necked T-shirt in heavy cotton jersey wi...,14,2,22,0.013542,0.050831,0.032341,3,8,9
2,2020-08-17,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,0904736002,0.033881,2,904736,Swirl hip belt,67,Belt,Accessories,1010016,Solid,9,Black,4,Dark,5,Black,3509,Belts,C,Ladies Accessories,1,Ladieswear,65,Womens Big accessories,1019,Accessories,Wide belt with a round metal buckle.,14,2,22,0.013542,0.050831,0.032341,3,8,9
3,2020-08-17,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,0904736002,0.033881,2,904736,Swirl hip belt,67,Belt,Accessories,1010016,Solid,9,Black,4,Dark,5,Black,3509,Belts,C,Ladies Accessories,1,Ladieswear,65,Womens Big accessories,1019,Accessories,Wide belt with a round metal buckle.,14,2,22,0.013542,0.050831,0.032341,3,8,9
4,2020-08-17,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,0903428001,0.025407,2,903428,Lazer LS Tee,255,T-shirt,Garment Upper body,1010014,Placement print,9,Black,4,Dark,5,Black,5882,Jersey Fancy,F,Menswear,3,Menswear,55,Contemporary Street,1005,Jersey Fancy,"Straight-style, long-sleeved top in soft cotto...",14,2,22,0.013542,0.050831,0.032341,3,8,9


In [17]:
# has to be run after above for train_df to exist
test_df = get_test_df()
test_df = clean_article_id(test_df)
test_df = add_features(test_df, use_train_df=True)

test_df.head()

train start date: 2020-08-17 00:00:00
test start date: 2020-09-16 00:00:00
test end date: 2020-09-23 00:00:00


,t_dat,customer_id,article_id,price,sales_channel_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,perceived_colour_value_id,perceived_colour_value_name,perceived_colour_master_id,perceived_colour_master_name,department_no,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc,nArticles,nDays,nPurchases,minPrice,maxPrice,meanPrice,nPGroups,nDept,nSect
0,2020-09-16,000fb6e772c5d0023892065e659963da90b1866035558e...,0786022008,0.048441,2.0,786022.0,Vivaldi dress,265.0,Dress,Garment Full body,1010016.0,Solid,43.0,Dark Red,4.0,Dark,18.0,Red,1313.0,Dress,A,Ladieswear,1.0,Ladieswear,11.0,Womens Tailoring,1013.0,Dresses Ladies,Knee-length dress in a patterned crêpe weave w...,13.0,5.0,15.0,0.016271,0.098136,0.044877,4.0,9.0,7.0
1,2020-09-16,000fb6e772c5d0023892065e659963da90b1866035558e...,0913272003,0.032288,2.0,913272.0,Monsieur jumper,252.0,Sweater,Garment Upper body,1010016.0,Solid,11.0,Off White,1.0,Dusty Light,9.0,White,1616.0,Knitwear,A,Ladieswear,1.0,Ladieswear,11.0,Womens Tailoring,1003.0,Knitwear,"Jumper in a soft, fine knit with decorative bu...",13.0,5.0,15.0,0.016271,0.098136,0.044877,4.0,9.0,7.0
2,2020-09-16,000fb6e772c5d0023892065e659963da90b1866035558e...,0889669006,0.056508,2.0,889669.0,Apollo barell leg trouser,272.0,Trousers,Garment Lower body,1010016.0,Solid,43.0,Dark Red,4.0,Dark,18.0,Red,1717.0,Trouser,A,Ladieswear,1.0,Ladieswear,11.0,Womens Tailoring,1009.0,Trousers,Ankle-length cigarette trousers in woven fabri...,13.0,5.0,15.0,0.016271,0.098136,0.044877,4.0,9.0,7.0
3,2020-09-18,000fb6e772c5d0023892065e659963da90b1866035558e...,0913272003,0.033881,2.0,913272.0,Monsieur jumper,252.0,Sweater,Garment Upper body,1010016.0,Solid,11.0,Off White,1.0,Dusty Light,9.0,White,1616.0,Knitwear,A,Ladieswear,1.0,Ladieswear,11.0,Womens Tailoring,1003.0,Knitwear,"Jumper in a soft, fine knit with decorative bu...",13.0,5.0,15.0,0.016271,0.098136,0.044877,4.0,9.0,7.0
4,2020-09-18,000fb6e772c5d0023892065e659963da90b1866035558e...,0889669006,0.059305,2.0,889669.0,Apollo barell leg trouser,272.0,Trousers,Garment Lower body,1010016.0,Solid,43.0,Dark Red,4.0,Dark,18.0,Red,1717.0,Trouser,A,Ladieswear,1.0,Ladieswear,11.0,Womens Tailoring,1009.0,Trousers,Ankle-length cigarette trousers in woven fabri...,13.0,5.0,15.0,0.016271,0.098136,0.044877,4.0,9.0,7.0


In [18]:
print('number of training samples:', len(train_df))
print('number of test samples:', len(test_df))

number of training samples: 1127168
number of test samples: 454324


In [19]:
print(psutil.virtual_memory().available * 100 / psutil.virtual_memory().total)

del all_data_df
gc.collect()

print(psutil.virtual_memory().available * 100 / psutil.virtual_memory().total)

91.28734689295574
94.7714349861973


In [20]:
# Remove records where we don't have all customer features
train_df = train_df[~train_df['customer_id'].isnull()]
test_df = test_df[~test_df['customer_id'].isnull()]

# Use 40k customers from train_df only so we can quickly test our methods
CUSTOMER_IDS = train_df['customer_id'].unique()[:40000]
train_df = train_df[train_df['customer_id'].isin(CUSTOMER_IDS)]
test_df = test_df[test_df['customer_id'].isin(CUSTOMER_IDS)]

In [21]:
print('number of training samples:', len(train_df))
print('number of test samples:', len(test_df))

number of training samples: 233096
number of test samples: 57309


## Generating negative samples

In [22]:
unique_articles = list(article_df['article_id'].unique())

In [23]:
def generate_negative_samples(
    df,
    unique_articles,
    num_neg=1
):
    # get a unique list of purchased articles for each customer
    customer_purchases_df = df.groupby("customer_id",).article_id.agg(['unique']).reset_index()
    customer_purchases_df.columns = ['customer_id', 'purchased_articles']
    
    customer_negative_samples = []
    
    for index, row in customer_purchases_df.iterrows():
        customer_id = row['customer_id']
        purchased_articles = row['purchased_articles']
        
        unpurchased_articles = list(set(unique_articles) - set(purchased_articles))
        
        negative_samples = random.sample(unpurchased_articles, num_neg)
        
        for article_id in negative_samples:
            customer_negative_samples.append(
                {
                    'customer_id': customer_id,
                    'article_id': article_id
                }
            )
    
    negative_samples_df = pd.DataFrame(customer_negative_samples)
    
    return negative_samples_df

In [24]:
train_neg_df = generate_negative_samples(train_df, unique_articles, num_neg=2)
test_neg_df = generate_negative_samples(test_df, unique_articles, num_neg=2)

train_neg_df.shape, test_neg_df.shape

((80000, 2), (80000, 2))

In [25]:
def combine_positive_and_negative_samples(
    positive_samples_df,
    negative_samples_df
):
    negative_samples_df = add_features(negative_samples_df, use_train_df=True)
    
    positive_samples_df['is_purchased'] = 1
    negative_samples_df['is_purchased'] = 0
    
    combined_df = pd.concat([positive_samples_df, negative_samples_df])
    
    return combined_df

In [26]:
combined_train_df = combine_positive_and_negative_samples(train_df, train_neg_df)
combined_test_df = combine_positive_and_negative_samples(test_df, test_neg_df)

## Building a single task model

In [27]:
numerical_features = ['nArticles','nDays','nPurchases','minPrice','maxPrice','meanPrice','nPGroups','nDept','nSect']
categorical_features = ['product_group_name']

In [100]:
X_numerical = combined_train_df[numerical_features].to_numpy()
X_categorical = combined_train_df[categorical_features].to_numpy().astype(str)

In [101]:
y_is_purchased = combined_train_df['is_purchased'].to_numpy()

In [102]:
inputs_numerical = tf.keras.Input((len(numerical_features)))
numerical_tower = tf.keras.layers.Dense(50, activation='relu')(inputs_numerical)

In [103]:
unique_product_group_names = article_df['product_group_name'].unique().astype(str)

In [104]:
inputs_categorical = tf.keras.Input((len(categorical_features)), dtype=tf.string)
categorical_tower = tf.keras.layers.StringLookup(vocabulary=unique_product_group_names, mask_token=None)(inputs_categorical)
categorical_tower = tf.keras.layers.Embedding(len(unique_product_group_names)+1, 50)(categorical_tower)
categorical_tower = tf.keras.layers.Dense(50, activation='relu')(categorical_tower)
categorical_tower = tf.keras.layers.Flatten()(categorical_tower)

In [105]:
combined_tower = tf.keras.layers.concatenate([numerical_tower, categorical_tower])
combined_tower = tf.keras.layers.Dense(50, activation='relu')(combined_tower)
output_is_purchased = tf.keras.layers.Dense(1, activation='sigmoid')(combined_tower)

In [137]:
single_task_model = tf.keras.Model([inputs_numerical, inputs_categorical], output_is_purchased)
single_task_model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1),
    metrics=[tf.keras.metrics.BinaryAccuracy()]
)

In [138]:
history = single_task_model.fit(
    [X_numerical, X_categorical],
    [y_is_purchased],
    epochs=10,
    batch_size=1000
)

Epoch 1/10
314/314 [==============================] - 3s 7ms/step - loss: 0.4724 - binary_accuracy: 0.7809
Epoch 2/10
314/314 [==============================] - 2s 7ms/step - loss: 0.4712 - binary_accuracy: 0.7818
Epoch 3/10
314/314 [==============================] - 2s 7ms/step - loss: 0.4709 - binary_accuracy: 0.7821
Epoch 4/10
314/314 [==============================] - 2s 7ms/step - loss: 0.4702 - binary_accuracy: 0.7824
Epoch 5/10
314/314 [==============================] - 2s 7ms/step - loss: 0.4699 - binary_accuracy: 0.7826
Epoch 6/10
314/314 [==============================] - 2s 7ms/step - loss: 0.4697 - binary_accuracy: 0.7829
Epoch 7/10
314/314 [==============================] - 2s 7ms/step - loss: 0.4696 - binary_accuracy: 0.7823
Epoch 8/10
314/314 [==============================] - 2s 7ms/step - loss: 0.4693 - binary_accuracy: 0.7825
Epoch 9/10
314/314 [==============================] - 2s 7ms/step - loss: 0.4692 - binary_accuracy: 0.7827
Epoch 10/10
314/314 [================

In [139]:
X_numerical_test = combined_test_df[numerical_features].to_numpy()
X_categorical_test = combined_test_df[categorical_features].to_numpy().astype(str)

predictions_is_purchased_test = single_task_model.predict(
    [X_numerical_test, X_categorical_test]
)
predictions_is_purchased_test = (predictions_is_purchased_test >= 0.5).astype(int).flatten()
sum(predictions_is_purchased_test == combined_test_df['is_purchased'].values)/len(predictions_is_purchased_test)

4291/4291 [==============================] - 10s 2ms/step


0.44894362350610667

## Building a multi-task model

In [95]:
def add_has_purchased_category_label(df):
    
    customer_category_purchases_df = train_df[['customer_id', 'product_group_name']].drop_duplicates()
    customer_category_purchases_df['has_purchased_category'] = 1
    
    df = df.join(
        customer_category_purchases_df.set_index(['customer_id', 'product_group_name']),
        on=['customer_id', 'product_group_name']
    )
    
    df['has_purchased_category'] = df['has_purchased_category'].fillna(0).astype(int)
    
    return df

In [97]:
combined_train_df = add_has_purchased_category_label(combined_train_df)
combined_test_df = add_has_purchased_category_label(combined_test_df)

In [140]:
numerical_features = ['nArticles','nDays','nPurchases','minPrice','maxPrice','meanPrice','nPGroups','nDept','nSect']
categorical_features = ['product_group_name']

In [141]:
X_numerical = combined_train_df[numerical_features].to_numpy()
X_categorical = combined_train_df[categorical_features].to_numpy().astype(str)

In [142]:
y_is_purchased = combined_train_df['is_purchased'].to_numpy()
y_has_purchased_category = combined_train_df['has_purchased_category'].to_numpy()

In [143]:
inputs_numerical = tf.keras.Input((len(numerical_features)))
numerical_tower = tf.keras.layers.Dense(64, activation='relu')(inputs_numerical)

In [144]:
unique_product_group_names = article_df['product_group_name'].unique().astype(str)

In [145]:
inputs_categorical = tf.keras.Input((len(categorical_features)), dtype=tf.string)
categorical_tower = tf.keras.layers.StringLookup(vocabulary=unique_product_group_names, mask_token=None)(inputs_categorical)
categorical_tower = tf.keras.layers.Embedding(len(unique_product_group_names)+1, 50)(categorical_tower)
categorical_tower = tf.keras.layers.Dense(64, activation='relu')(categorical_tower)
categorical_tower = tf.keras.layers.Flatten()(categorical_tower)

In [146]:
combined_tower = tf.keras.layers.concatenate([numerical_tower, categorical_tower])
combined_tower = tf.keras.layers.Dense(64, activation='relu')(combined_tower)

In [147]:
is_purchased_tower = tf.keras.layers.Dense(
    32, activation='relu', name='is_purchased_layer_1')(combined_tower)

is_purchased_tower = tf.keras.layers.Dense(
    16, activation='relu', name='is_purchased_layer_2')(is_purchased_tower)

output_is_purchased = tf.keras.layers.Dense(
    1, activation='sigmoid', name='is_purchased_output')(is_purchased_tower)

In [148]:
has_purchased_category_tower = tf.keras.layers.Dense(
    32, activation='relu', name='has_purchased_category_layer_1')(combined_tower)

has_purchased_category_tower = tf.keras.layers.Dense(
    16, activation='relu', name='has_purchased_category_layer_2')(has_purchased_category_tower)

output_has_purchased_category = tf.keras.layers.Dense(
    1, activation='sigmoid', name='has_purchased_category_output')(has_purchased_category_tower)

In [149]:
multi_task_model = tf.keras.Model(
    inputs=[inputs_numerical, inputs_categorical],
    outputs=[output_is_purchased, output_has_purchased_category]
)

multi_task_model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1),
    metrics=[tf.keras.metrics.BinaryAccuracy()]
)

In [150]:
history = multi_task_model.fit(
    [X_numerical, X_categorical],
    [y_is_purchased, y_has_purchased_category],
    epochs=10,
    batch_size=1000
)

Epoch 1/10
314/314 [==============================] - 4s 8ms/step - loss: 0.8590 - is_purchased_output_loss: 0.5212 - has_purchased_category_output_loss: 0.3377 - is_purchased_output_binary_accuracy: 0.7436 - has_purchased_category_output_binary_accuracy: 0.8550
Epoch 2/10
314/314 [==============================] - 2s 8ms/step - loss: 0.8022 - is_purchased_output_loss: 0.4956 - has_purchased_category_output_loss: 0.3067 - is_purchased_output_binary_accuracy: 0.7671 - has_purchased_category_output_binary_accuracy: 0.8684
Epoch 3/10
314/314 [==============================] - 2s 8ms/step - loss: 0.7900 - is_purchased_output_loss: 0.4882 - has_purchased_category_output_loss: 0.3018 - is_purchased_output_binary_accuracy: 0.7730 - has_purchased_category_output_binary_accuracy: 0.8708
Epoch 4/10
314/314 [==============================] - 2s 7ms/step - loss: 0.7832 - is_purchased_output_loss: 0.4842 - has_purchased_category_output_loss: 0.2990 - is_purchased_output_binary_accuracy: 0.7741 - ha

In [151]:
X_numerical_test = combined_test_df[numerical_features].to_numpy()
X_categorical_test = combined_test_df[categorical_features].to_numpy().astype(str)

predictions_is_purchased_test = multi_task_model.predict(
    [X_numerical_test, X_categorical_test]
)
predictions_is_purchased_test = (predictions_is_purchased_test[0] >= 0.5).astype(int).flatten()
sum(predictions_is_purchased_test == combined_test_df['is_purchased'].values)/len(predictions_is_purchased_test)

4291/4291 [==============================] - 11s 3ms/step


0.546351659395961

### Freezing layers

In [156]:
for layer in multi_task_model.layers:
    print(f'{layer.name}, trainable: {layer.trainable}')

input_10, trainable: True
string_lookup_4, trainable: True
embedding_4, trainable: True
input_9, trainable: True
dense_31, trainable: True
dense_30, trainable: True
flatten_4, trainable: True
concatenate_4, trainable: True
dense_32, trainable: True
is_purchased_layer_1, trainable: True
has_purchased_category_layer_1, trainable: True
is_purchased_layer_2, trainable: True
has_purchased_category_layer_2, trainable: True
is_purchased_output, trainable: True
has_purchased_category_output, trainable: True


In [157]:
for layer in multi_task_model.layers:
    if layer.name in [
        'has_purchased_category_layer_1',
        'has_purchased_category_layer_2',
        'has_purchased_category_output'
    ]:
        layer.trainable = False

In [158]:
for layer in multi_task_model.layers:
    print(f'{layer.name}, trainable: {layer.trainable}')

input_10, trainable: True
string_lookup_4, trainable: True
embedding_4, trainable: True
input_9, trainable: True
dense_31, trainable: True
dense_30, trainable: True
flatten_4, trainable: True
concatenate_4, trainable: True
dense_32, trainable: True
is_purchased_layer_1, trainable: True
has_purchased_category_layer_1, trainable: False
is_purchased_layer_2, trainable: True
has_purchased_category_layer_2, trainable: False
is_purchased_output, trainable: True
has_purchased_category_output, trainable: False


In [159]:
multi_task_model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1),
    metrics=[tf.keras.metrics.BinaryAccuracy()]
)

In [160]:
history = multi_task_model.fit(
    [X_numerical, X_categorical],
    [y_is_purchased, y_has_purchased_category],
    epochs=10,
    batch_size=1000
)

Epoch 1/10
314/314 [==============================] - 4s 8ms/step - loss: 0.7757 - is_purchased_output_loss: 0.4811 - has_purchased_category_output_loss: 0.2946 - is_purchased_output_binary_accuracy: 0.7762 - has_purchased_category_output_binary_accuracy: 0.8720
Epoch 2/10
314/314 [==============================] - 2s 7ms/step - loss: 0.7651 - is_purchased_output_loss: 0.4749 - has_purchased_category_output_loss: 0.2902 - is_purchased_output_binary_accuracy: 0.7794 - has_purchased_category_output_binary_accuracy: 0.8736
Epoch 3/10
314/314 [==============================] - 2s 7ms/step - loss: 0.7637 - is_purchased_output_loss: 0.4742 - has_purchased_category_output_loss: 0.2895 - is_purchased_output_binary_accuracy: 0.7796 - has_purchased_category_output_binary_accuracy: 0.8740
Epoch 4/10
314/314 [==============================] - 2s 7ms/step - loss: 0.7620 - is_purchased_output_loss: 0.4733 - has_purchased_category_output_loss: 0.2888 - is_purchased_output_binary_accuracy: 0.7805 - ha

# Contextual Bandits

## Setting up

In [16]:
import numpy as np
import fileinput
import random
import time

## Getting data

In [9]:
files = ('../data/yahoo/data.txt')

In [10]:
def read_data(filenames):
    """
    Reads a stream of events from the list of given files.
    
    Parameters
    ----------
    filenames : list
        List of filenames
    
    Stores
    -------    
    articles : [article_ids]
    features : [[article_1_features] .. [article_n_features]]
    events : [
                 0 : displayed_article_index (relative to the pool),
                 1 : user_click,
                 2 : [user_features],
                 3 : [pool_indexes]
             ]
    """

    global articles, features, events, n_arms, n_events
    articles = []
    features = []
    events = []

    skipped = 0

    with fileinput.input(files=filenames) as f:
        for line in f:
            cols = line.split()
            if (len(cols) - 10) % 7 != 0:
                skipped += 1
            else:
                pool_idx = []
                pool_ids = []
                for i in range(10, len(cols) - 6, 7):
                    id = cols[i][1:]
                    if id not in articles:
                        articles.append(id)
                        features.append([float(x[2:]) for x in cols[i + 1: i + 7]])
                    pool_idx.append(articles.index(id))
                    pool_ids.append(id)

                events.append(
                    [
                        pool_ids.index(cols[1]),
                        int(cols[2]),
                        [float(x[2:]) for x in cols[4:10]],
                        pool_idx,
                    ]
                )
    features = np.array(features)
    n_arms = len(articles)
    n_events = len(events)
    print(n_events, "events with", n_arms, "articles")
    if skipped != 0:
        print("Skipped events:", skipped)

In [11]:
read_data(files)

100000 events with 21 articles


## Evaluating policies

In [17]:
def evaluate(A, size=100, learn_ratio = 0.9):
    '''
    Policy evaluator as described in the paper
    Parameters
    ----------
    A : class (algorithm)
    size : number (run the evaluation only on a portion of the dataset)
    learn_ratio : number (perform learning(update parameters) only on a small portion of the traffic)
    Returns
    -------
    learn : array (contains the ctr for each trial for the learning bucket)
    deploy : array (contains the ctr for each trial for the deployment bucket)
    '''
    
    start = time.time()
    # we initialize the payoff and events parameters separately for learning phase of the events and deployment phase of events.
    Payoff_deploy = 0 # total payoff for the deployment bucket
    Payoff_learn = 0  # total payoff for the learning bucket
    Events_deploy = 1 # counter of valid events for the deployment bucket
    Events_learn = 0  # counter of valid events for the learning bucket

    learn = []
    deploy = []
    global events
    if size != 100:
        k = int(n_events * size / 100)
        events = random.sample(events, k)

    '''
    we run through the logged events, and treat each event either for learning & updating the parameters,
    or for deployment purposes wherein we use the reward obtained as evaluation metric
    '''
    for t, event in enumerate(events):

        displayed = event[0]
        reward = event[1]
        user = event[2]
        pool_idx = event[3]

        # select the arm based on the bandit policy
        chosen = A.choose_arm(Payoff_learn + Payoff_deploy, user, pool_idx)
        
        '''
        If, given the current history ht−1, it happens that the policy A chooses the same arm a
        as the one that was selected by the logging policy, then the event is retained
        (that is, added to the history), and the total payoff updated.
        Otherwise, if the policy A selects a different arm from the one that was taken by the logging policy,
        then the event is entirely ignored, and the algorithm proceeds to the next event without any change in its state.
        '''
        if chosen == displayed:
            if random.random() < learn_ratio:
                Payoff_learn += event[1]
                Events_learn += 1
                A.update(displayed, reward, user, pool_idx)
                learn.append(Payoff_learn / Events_learn)
            else:
                Payoff_deploy += event[1]
                Events_deploy += 1
                deploy.append(Payoff_deploy / Events_deploy)

    end = time.time()

    execution_time = round(end - start, 1)
    execution_time = (
        str(round(execution_time / 60, 1)) + 'm'
        if execution_time > 60
        else str(execution_time) + 's'
    )
    print(
        '{:<20}{:<10}{}'.format(
            A.algorithm, round(Payoff_deploy / Events_deploy, 4), execution_time
        )
    )

    return learn, deploy

## ϵ-greedy

In [15]:
class Egreedy:
    '''
    Epsilon greedy algorithm implementation
    '''

    def __init__(self, epsilon):
        '''
        Parameters
        ----------
        epsilon : number (Egreedy parameter, ideally between 0 and 1)
        '''

        self.e = round(epsilon, 1)  # epsilon parameter for Egreedy 
        self.algorithm = f'Egreedy (ε={self.e})'
        self.q = np.zeros(n_arms)  # average reward for each arm -- this represents the known mean reward for each arm
        self.n = np.zeros(n_arms)  # number of times each arm was chosen

    def choose_arm(self, t, user, pool_idx):
        '''
        Returns the best arm's index relative to the pool
        Parameters
        ----------
        t : number (number of trial)
        user : array (user features)
        pool_idx : array of indexes (pool indexes for article identification)
        '''

        p = np.random.rand()
        if p > self.e:
            return np.argmax(self.q[pool_idx])
        else:
            return np.random.randint(low=0, high=len(pool_idx))

    def update(self, displayed, reward, user, pool_idx):
        '''
        Updates algorithm's parameters(matrices)
        Parameters
        ----------
        displayed : index (displayed article index relative to the pool)
        reward : binary (user clicked or not)
        user : array (user features)
        pool_idx : array of indexes (pool indexes for article identification)
        '''

        a = pool_idx[displayed]
        
        # update counts pulled for chosen arm
        self.n[a] += 1
        
        # update average/mean value/reward for chosen arm
        self.q[a] += (reward - self.q[a]) / self.n[a]
        '''
        this can also be written as:
        value = self.q[a]
        new_value = ((self.n[a]-1)/float(self.n[a])) * value + (1 / float(self.n[a])) * reward
        self.q[a] = new_value
        '''

In [24]:
_, deploy = evaluate(Egreedy(0.1),learn_ratio=0.99)
rnd_ctr = deploy[-1]

_, deploy = evaluate(Egreedy(0.5),learn_ratio=0.99)
rnd_ctr = deploy[-1]

_, deploy = evaluate(Egreedy(0.9),learn_ratio=0.99)
rnd_ctr = deploy[-1]

Egreedy (ε=0.1)     0.0444    1.1s
Egreedy (ε=0.5)     0.0222    0.9s
Egreedy (ε=0.9)     0.0323    0.7s


## UCB-1

In [36]:
class Ucb1:
    def __init__(self, alpha):
        '''
        Parameters
        ----------
        alpha : number (ucb parameter)
        '''

        self.alpha = round(alpha, 1)
        self.algorithm = f'UCB1 (α={self.alpha})'

        self.q = np.zeros(n_arms)  # average reward for each arm
        self.n = np.ones(n_arms)  # number of times each arm was chosen

    def choose_arm(self, t, user, pool_idx):
        '''
        Returns the best arm's index relative to the pool
        Parameters
        ----------
        t : number (number of trial)
        user : array (user features)
        pool_idx : array of indexes (pool indexes for article identification)
        '''
        
        upper_bounds = self.q[pool_idx] + np.sqrt( self.alpha*np.log(t+1) / self.n[pool_idx] )
        
        return np.argmax(upper_bounds)

    def update(self, displayed, reward, user, pool_idx):
        '''
        Updates algorithm's parameters(matrices)
        Parameters
        ----------
        displayed : index (displayed article index relative to the pool)
        reward : binary (user clicked or not)
        user : array (user features)
        pool_idx : array of indexes (pool indexes for article identification)
        '''

        a = pool_idx[displayed]
        
        # update counts pulled for chosen arm
        self.n[a] += 1
        
        # update average/mean value/reward for chosen arm
        self.q[a] += (reward - self.q[a]) / self.n[a]


In [37]:
_, deploy = evaluate(Ucb1(2),learn_ratio=0.99)
rnd_ctr = deploy[-1]

UCB1 (α=2)          0.0488    2.5s
